In [ ]:
from fenics import *
%matplotlib inline
from mshr import *
import matplotlib.pyplot as plt

In [ ]:
C = Circle(Point(0,0), 1)
mesh = generate_mesh(C,16)
#mesh = UnitSquareMesh(16, 16)
plot(mesh)

In [ ]:
# using mesh, function type, degree, how many component
V = VectorFunctionSpace(mesh, "P", 1, 2)

In [ ]:
u = Function(V)
un = Function(V)
v = TestFunction(V)

u1, u2 = split(u)
u1n, u2n = split(un)
v1, v2 = split(v)

In [ ]:
eps = Constant(0.1)
dt = 0.01
k = Constant(dt)
K = Constant(10)

x = SpatialCoordinate(mesh)
beta = as_vector((-2.0*x[1], 2.0*x[0])) # Or you can use expression

f1 = Expression('exp(-50.0*(pow(x[0]+0.75, 2)+ pow(x[1], 2)))', degree=2)
f2 = Expression('exp(-50.0*(pow(x[0]-0.75, 2)+ pow(x[1], 2)))', degree=2)

F = (((u1- u1n)/k)*v1 + dot(beta, grad(u1))*v1 + \
    eps*dot(grad(u1), grad(v1)))*dx +\
    (((u2- u2n)/k)*v2 + dot(beta, grad(u2))*v2 + \
    eps*dot(grad(u2), grad(v2)))*dx +\
    - (f1*v1 +f2*v2 - K*u1*u2**2*v1 - 2*K*u1*u2**2*v2)*dx

In [ ]:
num_steps = 100
t = 0.0
for n in range(num_steps):
    # Update current time
    t += dt
    print("t = ", t)
    
    # Solve variational problem
    solve(F == 0, u)
    
    # Save solution to file
    _u1, _u2 = u.split()

    p_1 = plot(_u1)
    p_1.set_cmap("seismic")
    plt.title("U_1")
    plt.colorbar(p_1)   
    plt.show()     
    
    
    p_2 = plot(_u2)
    p_2.set_cmap("seismic")
    plt.title("U_2")
    plt.colorbar(p_2)   
    plt.show() 
#    file_u1 << _u1
#    file_u2 << _u2
    
    # Update previous solution
    un.assign(u)